### This notebook will contain the loading of the datasets, and tie together the different parts of the project

### IMPORTS

In [1]:
import numpy as np
import pandas as pd
from PriceDataLoader import PriceDataLoader
from WeatherDataLoader import WeatherDataLoader
from models.CNN import *
from models.FFN import *
import torch
import torch.nn as nn
from models.CointegrationResidualGenerator import CointegrationResidualGenerator
from models.BacktestSharpeEvaluator import BacktestSharpeEvaluator
from PortfolioOptimizer import PortfolioOptimizer
from TimeSeriesDataSplitter import TimeSeriesDataSplitter
from Utility import Utility


### Data Preperation


In [20]:
# Initialize the DataLoader with the test data
price_parser = PriceDataLoader(file_path="../data/european_wholesale_electricity_price_data_daily.csv")

# Get list of countries
all_countries_list = price_parser.get_country_list()
small_counties_list = all_countries_list[:5]

# Define the total time range
train_time_range = "2015-01-01,2024-01-01"

# Define the window size and stride
one_window_days = 365
window_stride_days = 5

# Get the rolling window of the test data
x_data = price_parser.get_price_matrix_rolling_window(
    one_window_days=one_window_days,
    window_stride_days=window_stride_days,
    time_range=train_time_range,
    countries=all_countries_list,
    fill_method="ffill",
)

# -----------------------------------------------------

# Get the price matrix for the training/testing/validation data
price_matrix = price_parser.get_price_matrix(
    time_range=train_time_range,
    countries=all_countries_list,
    fill_method="ffill",
)

# ------------------------------------------------------
# Get the returns matrix
returns_tr = price_matrix.pct_change().dropna()


Data loaded successfully from ../data/european_wholesale_electricity_price_data_daily.csv


In [21]:
# Get the rolling window of the training data
x_cumulative_residuals, y_next_day_returns = Utility.process_training_data(
    x_tr_data=x_data,
    returns=returns_tr,
    cumulative_residual_window=30
)

# Convert x_tr_cumulative_residuals to a numpy array
x_cumulative_residuals = np.array(x_cumulative_residuals)

Skipping due to error: Found array with 0 sample(s) (shape=(0, 30)) while a minimum of 1 is required by LinearRegression.
Skipping due to error: Found array with 0 sample(s) (shape=(0, 30)) while a minimum of 1 is required by LinearRegression.
Skipping due to error: Found array with 0 sample(s) (shape=(0, 30)) while a minimum of 1 is required by LinearRegression.
Skipping due to error: Found array with 0 sample(s) (shape=(0, 30)) while a minimum of 1 is required by LinearRegression.
Skipping due to error: Found array with 0 sample(s) (shape=(0, 30)) while a minimum of 1 is required by LinearRegression.
Skipping due to error: Found array with 0 sample(s) (shape=(0, 30)) while a minimum of 1 is required by LinearRegression.
Skipping due to error: Found array with 0 sample(s) (shape=(0, 30)) while a minimum of 1 is required by LinearRegression.
Skipping due to error: Found array with 0 sample(s) (shape=(0, 30)) while a minimum of 1 is required by LinearRegression.
Skipping due to error: F

/Users/filip/stat7007/project/venv/lib/python3.12/site-packages/pandas/core/internals/blocks.py:393: RuntimeWarning: invalid value encountered in log
  result = func(self.values, **kwargs)
/Users/filip/stat7007/project/venv/lib/python3.12/site-packages/pandas/core/internals/blocks.py:393: RuntimeWarning: invalid value encountered in log
  result = func(self.values, **kwargs)
/Users/filip/stat7007/project/venv/lib/python3.12/site-packages/pandas/core/internals/blocks.py:393: RuntimeWarning: invalid value encountered in log
  result = func(self.values, **kwargs)
/Users/filip/stat7007/project/venv/lib/python3.12/site-packages/pandas/core/internals/blocks.py:393: RuntimeWarning: invalid value encountered in log
  result = func(self.values, **kwargs)
/Users/filip/stat7007/project/venv/lib/python3.12/site-packages/pandas/core/internals/blocks.py:393: RuntimeWarning: invalid value encountered in log
  result = func(self.values, **kwargs)
/Users/filip/stat7007/project/venv/lib/python3.12/site-

The cumulative residual window size exceeds the available data.


/Users/filip/stat7007/project/venv/lib/python3.12/site-packages/pandas/core/internals/blocks.py:393: RuntimeWarning: invalid value encountered in log
  result = func(self.values, **kwargs)
/Users/filip/stat7007/project/venv/lib/python3.12/site-packages/pandas/core/internals/blocks.py:393: RuntimeWarning: invalid value encountered in log
  result = func(self.values, **kwargs)
/Users/filip/stat7007/project/venv/lib/python3.12/site-packages/pandas/core/internals/blocks.py:393: RuntimeWarning: invalid value encountered in log
  result = func(self.values, **kwargs)
/Users/filip/stat7007/project/venv/lib/python3.12/site-packages/pandas/core/internals/blocks.py:393: RuntimeWarning: invalid value encountered in log
  result = func(self.values, **kwargs)
/Users/filip/stat7007/project/venv/lib/python3.12/site-packages/pandas/core/internals/blocks.py:393: RuntimeWarning: invalid value encountered in log
  result = func(self.values, **kwargs)
/Users/filip/stat7007/project/venv/lib/python3.12/site-

In [22]:
# # Print the shape of the cnn_input_array
# print(f"cnn_input_array shape: {cnn_input_array.shape}")
# # Print the type of the cnn_input_array
# print(f"cnn_input_array type: {type(cnn_input_array)}")

# # Print the shape of next_day_returns
# print(f"next_day_returns shape: {next_day_returns.shape}")
# # Print the type of next_day_returns
# print(f"next_day_returns type: {type(next_day_returns)}")

print("------------------------------------")

# Print the shape of a3_0
print(f"x_cumulative_residuals shape: {x_cumulative_residuals.shape}")
# Print the type of a3_0
print(f"x_cumulative_residuals type: {type(x_cumulative_residuals)}")

# Print the shape of a3_1
print(f"y_next_day_returns shape: {y_next_day_returns.shape}")
# Print the type of a3_1
print(f"y_next_day_returns type: {type(y_next_day_returns)}")

------------------------------------
x_cumulative_residuals shape: (111779, 31, 30)
x_cumulative_residuals type: <class 'numpy.ndarray'>
y_next_day_returns shape: (111779, 31)
y_next_day_returns type: <class 'pandas.core.frame.DataFrame'>


### CNN + FFN (Ablation study)

#### Sharpe ratio

In [5]:
def sharpe_ratio_loss(returns, risk_free_rate=0.0):
    """
    Custom loss function to maximize the Sharpe Ratio.
    Args:
        returns (torch.Tensor): Predicted returns
        risk_free_rate (float): Risk-free rate for Sharpe calculation
    Returns:
        torch.Tensor: Negative Sharpe Ratio (to minimize)
    """
    excess_returns = returns - risk_free_rate
    mean_excess = torch.mean(excess_returns)
    std_excess = torch.std(excess_returns, unbiased=False) + 1e-6  # epsilon for stability
    sharpe_ratio = mean_excess / std_excess
    return -sharpe_ratio

In [16]:
### -------------- PARSING DATA -------------- ###
price_parser = PriceDataLoader(file_path="../data/european_wholesale_electricity_price_data_daily.csv")
weather_parser = WeatherDataLoader(file_path="../data/aggregated_weather.csv")

# Get list of countries
all_countries_list = price_parser.get_country_list()
print(f"Number of countries: {len(all_countries_list)}")

# Get daily price matrix for all countries for 2021
price_matrix = price_parser.get_price_matrix(
    time_range="2021-01-01,2021-12-31",
    countries=all_countries_list,
    fill_method="ffill"
)

# Get daily returns
returns = price_matrix.pct_change().dropna()

# Get weather matrix with selected features
weather_features = ['temperature_2m_mean', 'wind_speed_mean']  # Adjust based on available features

print(f"Number of features: {len(weather_features) + 1}") # +1 due to price

weather_matrix = weather_parser.get_weather_matrix(
    time_range="2021-01-01,2021-12-31",
    countries=all_countries_list,
    fill_method="ffill",
    features=weather_features
)

### -------------- COINTEGRATION RESIDUALS -------------- ###
residual_generator = CointegrationResidualGenerator(price_matrix)
residual_generator.compute_all_asset_residuals()
asset_residuals = residual_generator.get_asset_residuals()

# Prepare CNN input from residuals
cumulative_residual_window = 30
cnn_residual_input = residual_generator.prepare_cnn_input_from_residuals(window=cumulative_residual_window)

# Prepare weather input with rolling windows
weather_rolling_windows = weather_parser.get_weather_matrix_rolling_window(
    one_window_days=cumulative_residual_window,
    window_stride_days=1,
    time_range="2021-01-01,2021-12-31",
    countries=all_countries_list,
    fill_method="ffill",
    features=weather_features
)
print(f"Weather rolling window shape: {len(weather_rolling_windows)} x {len(weather_rolling_windows[0])}")

# Ensure alignment between residual and weather windows
# cnn_residual_input shape: [samples, window, assets]
# weather_rolling_windows: List of DataFrames, each [window, (feature, country)]
num_samples = len(cnn_residual_input)
combined_input = []
for i in range(num_samples):
    if i < len(weather_rolling_windows):
        weather_window = weather_rolling_windows[i]
        # Flatten MultiIndex columns to [country_feature]
        weather_array = weather_window.values  # Shape: [window, num_features * num_countries]
        residual_array = cnn_residual_input[i]  # Shape: [window, num_countries]
        # Concatenate residuals and weather features along feature dimension
        combined_window = np.concatenate([residual_array, weather_array], axis=1)  # Shape: [window, num_countries + num_features * num_countries]
        combined_input.append(combined_window)
combined_input_array = np.array(combined_input)  # Shape: [samples, window, num_countries + num_features * num_countries]

# Get next-day returns
start_idx_in_returns = returns.index.get_loc(asset_residuals.index[0])
next_day_indices = [start_idx_in_returns + i + cumulative_residual_window for i in range(num_samples)]
next_day_returns = returns.iloc[next_day_indices]

# Transform combined input for CNN
cnn_input_array = combined_input_array.transpose(0, 2, 1)  # [samples, features, window]

print(cnn_input_array.shape)

Data loaded successfully from ../data/european_wholesale_electricity_price_data_daily.csv
Weather data loaded successfully from ../data/aggregated_weather.csv
Number of countries: 31
Number of features: 3
Weather rolling window shape: 336 x 30
(329, 89, 30)


/Users/filip/stat7007/project/venv/lib/python3.12/site-packages/pandas/core/internals/blocks.py:393: RuntimeWarning: invalid value encountered in log
  result = func(self.values, **kwargs)


In [ ]:
### -------------- FEED THE 30-DAY CUMULATIVE RESIDUALS OF EVERY COUNTRY TO CNN+FFN -------------- ###
# Transform cnn_input to be compatible with the CNN input shape
#cnn_input_array = cnn_input.transpose(0, 2, 1) # [samples, features, window]

# FILIP'S SECTION: CNN+FFN
# Hey Filip, this is the section where you can add your code to train the CNN+FFN model.
# cnn_input_array essentially contains 329 training data points, each data point is 30-day cumulative residuals for the 31 countries.
# So, for one "data point" you would feed the set of 30-day cumulative residuals for every country to the CNN+FFN model.
# One data point should result in a set of weights for each of the 31 countries.
# Each set of weights is used to calculate one next-day portfolio return.
# Repeat this for all 329 data points to get a set of 329 portfolio returns.
# You can then use these portfolio returns to calculate the Sharpe ratio
# Optimize the CNN+FFN to maximize the Sharpe ratio.

# Set the device to GPU if available, otherwise MPS (for Mac silicon) or CPU
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

torch.manual_seed(1)  # For reproducibility

# Create the PortfolioOptimizer instance and train the model
# NOTE: We need to have some validation data to evaluate the model performance
# otherwise we will be overfitting the model to the training data

# Grid search for hyperparameters
num_filters = [8, 16, 32]
filter_sizes = [3, 5, 7]
hidden_dims = [64, 128]

# Choose the best hyperparameters based on validation performance
best_score = float('-inf')
best_params = None

# Training and validation split
# 70% training, 15% validation, 15% testing
# Note: The test set is not used in the training process, but it can be used ONCE to evaluate the final model performance
# The validation set is used to tune the hyperparameters and avoid overfitting
"""
train_dataset = cnn_input_array[:int(0.7 * len(cnn_input_array))]
train_next_day_returns = next_day_returns[:int(0.7 * len(next_day_returns))]

val_dataset = cnn_input_array[int(0.7 * len(cnn_input_array)):]
val_next_day_returns = next_day_returns[int(0.7 * len(next_day_returns)):]

test_dataset = val_dataset[int(0.5 * len(val_dataset)):]
test_next_day_returns = val_next_day_returns[int(0.5 * len(val_next_day_returns)):]
"""
# datasetSplitter = TimeSeriesDataSplitter(data_x=cnn_input_array, data_y=next_day_returns, train_size=0.7, validation_size=0.15, test_size=0.15)
datasetSplitter = TimeSeriesDataSplitter(data_x=x_cumulative_residuals, data_y=y_next_day_returns, train_size=0.7, validation_size=0.15, test_size=0.15)
(train), (val), (test) = datasetSplitter.split_data()

# Unpack the datasets
train_dataset, train_next_day_returns = train
val_dataset, val_next_day_returns = val
test_dataset, test_next_day_returns = test

print(f"Train dataset shape: {train_dataset.shape}, Train next day returns shape: {train_next_day_returns.shape}")
print(f"Validation dataset shape: {val_dataset.shape}, Validation next day returns shape: {val_next_day_returns.shape}")
print(f"Test dataset shape: {test_dataset.shape}, Test next day returns shape: {test_next_day_returns.shape}")

for num_filter in num_filters:
    for filter_size in filter_sizes:
        for hidden_dim in hidden_dims:
                print(f"Training with filters: {num_filter}, filter size: {filter_size}, hidden dim: {hidden_dim}")
                
                # Initialize the PortfolioOptimizer
                optimizer = PortfolioOptimizer(train_dataset, 
                                               train_next_day_returns, 
                                               val_dataset,
                                               val_next_day_returns,
                                               batch_size=1000, 
                                               num_epochs=1000, 
                                               num_filters=num_filter, 
                                               device=device, 
                                               filter_size=filter_size,
                                               hidden_dim=hidden_dim,
                                               patience=20,
                                               use_transformer=True)
                
                # Train the model
                _, portfolio_returns = optimizer.train(verbose=True) # The final sharpe should be from the test set

                # Evaluate the model's hyperparameters on the validation set
                val_sharpe, _ = optimizer.evaluate_final()
                if val_sharpe > best_score:
                    best_score = val_sharpe
                    best_params = (num_filter, filter_size, hidden_dim)
                    best_portfolio_returns = portfolio_returns
    
print(f"Best parameters: {best_params} with Sharpe Ratio on Validation Set: {best_score}")

### -------------- GET THE WEIGHTS OUTPUTTED FROM CNN+FFN -------------- ###
# Initializing the Sharpe ratio evaluator
#evaluator = BacktestSharpeEvaluator()

# Get the weight outputted from the CNN+FFN model
# One weight for each country
#weights = np.array([]) # CHANGE THIS TO THE ACTUAL WEIGHTS OUTPUTTED FROM THE CNN+FFN MODEL

# Normalize the weights using L1 normalization
# This is done to ensure that the portfolio is dollar-neutral
#normalized_weights = evaluator.normalize_weights_l1(weights)

# Multiply the normalized weights (vector) with the next-day returns (vector) to get the portfolio return
#next_day_portfolio_return = evaluator.compute_portfolio_return(normalized_weights, next_day_returns.iloc[0].values)

# Store the next day portfolio return 
#evaluator.add_return(next_day_portfolio_return)

# Repeat the above step for all 329 data points, adding the portfolio returns to the evaluator for each data point

# Once all portfolio returns are calculated, you can calculate the Sharpe ratio

# Train the model to optimize the Sharpe Ratio

In [ ]:
from Trainer import Trainer
from PortfolioOptimizer import PortfolioOptimizer
from DataPreparation import DataPreparation

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else
                        "mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using device: {device}")

# 1. Load and prepare data
price_parser = PriceDataLoader(file_path="../data/european_wholesale_electricity_price_data_daily.csv")
weather_parser = WeatherDataLoader(file_path="../data/aggregated_weather.csv")
all_countries_list = price_parser.get_country_list()
print(f"Number of countries: {len(all_countries_list)}")

# Date range
time_range = "2015-01-01,2024-12-31"
price_matrix = price_parser.get_price_matrix(time_range=time_range, countries=all_countries_list, fill_method="ffill")
returns = price_matrix.pct_change().dropna()
weather_features = ['temperature_2m_mean', 'temperature_2m_min', 'temperature_2m_max','precipitation_mean', 'precipitation_min','precipitation_max', 'wind_speed_mean', 'wind_speed_min', 'wind_speed_max']
print(f"Number of total features: {len(weather_features) + 1}")

weather_matrix = weather_parser.get_weather_matrix(
    time_range=time_range,
    countries=all_countries_list,
    fill_method="ffill",
    features=weather_features
)

# 2. Generate cointegration residuals
residual_generator = CointegrationResidualGenerator(price_matrix)
residual_generator.compute_all_asset_residuals()
asset_residuals = residual_generator.get_asset_residuals() # [num_days, num_assets (= countries)]

# 3. Prepare data
data_prep = DataPreparation(
    price_residuals=asset_residuals,
    weather_data=weather_matrix,
    countries=all_countries_list,
    weather_features=weather_features
)
window_size = 30
stride = 1
combined_data = data_prep.prepare_rolling_windows(window_size=window_size, stride=stride)
# [num_samples, num_features, window_size]

next_day_returns = data_prep.prepare_next_day_returns(returns=returns, window_size=window_size, stride=stride)
# [num_samples, num_countries]

(train_data, train_returns), (val_data, val_returns), (test_data, test_returns) = \
    data_prep.create_train_val_test_split(combined_data=combined_data, next_day_returns=next_day_returns)

# Convert to tensors
train_data_tensor = torch.FloatTensor(train_data).to(device)
train_returns_tensor = torch.FloatTensor(train_returns).to(device)
val_data_tensor = torch.FloatTensor(val_data).to(device)
val_returns_tensor = torch.FloatTensor(val_returns).to(device)
test_data_tensor = torch.FloatTensor(test_data).to(device)
test_returns_tensor = torch.FloatTensor(test_returns).to(device)

print(f"Train data shape: {train_data_tensor.shape}")
print(f"Train returns shape: {train_returns_tensor.shape}")

# 4. Initialize portfolio optimizer
portfolio_optimizer = PortfolioOptimizer(
    window_size=window_size,
    num_countries=len(all_countries_list),
    num_weather_features=len(weather_features),
    num_filters=8,  # Default value
    filter_size=3,
    hidden_dim=64,
    num_heads=4,
    use_transformer=True,
    device=device
)

# 5. Set up trainer with grid search
trainer = Trainer(
    optimizer=portfolio_optimizer,
    train_data=train_data_tensor,
    train_returns=train_returns_tensor,
    val_data=val_data_tensor,
    val_returns=val_returns_tensor,
    lr=0.001,
    num_epochs=100,
    batch_size=32,
    patience=50,
    device=device,
)

# 6. Perform grid search
param_grid = {
    'num_filters': [8, 16, 32],
    'filter_size': [3, 5, 7],
    'hidden_dim': [64, 128]
}
best_params, best_score, best_returns = trainer.grid_search(param_grid, verbose=False)
print(f"Best parameters: {best_params} with Sharpe Ratio: {best_score:.4f}")

Using device: mps
Data loaded successfully from ../data/european_wholesale_electricity_price_data_daily.csv
Weather data loaded successfully from ../data/aggregated_weather.csv
Number of countries: 31
Number of total features: 10


/Users/filip/stat7007/project/venv/lib/python3.12/site-packages/pandas/core/internals/blocks.py:393: RuntimeWarning: invalid value encountered in log
  result = func(self.values, **kwargs)


(2181, 31)


KeyboardInterrupt: 

In [ ]:
# 7. Reinitialize with best parameters and train on train+val
portfolio_optimizer = PortfolioOptimizer(
    window_size=window_size,
    num_countries=len(all_countries_list),
    num_weather_features=len(weather_features),
    num_filters=best_params['num_filters'],
    filter_size=best_params['filter_size'],
    hidden_dim=best_params['hidden_dim'],
    num_heads=4,
    use_transformer=True,
    device=device
)
trainer = Trainer(
    optimizer=portfolio_optimizer,
    train_data=torch.cat([train_data_tensor, val_data_tensor]),  # Combine train and val
    train_returns=torch.cat([train_returns_tensor, val_returns_tensor]),
    val_data=val_data_tensor,  # Still use val for early stopping
    val_returns=val_returns_tensor,
    lr=0.001,
    num_epochs=100,
    batch_size=32,
    patience=50,
    device=device
)
final_sharpe, _ = trainer.train(verbose=True)

# 8. Evaluate on test set
test_sharpe, test_returns = trainer.test(test_data_tensor, test_returns_tensor)
print(f"Final Sharpe Ratio on Test Set: {test_sharpe:.4f}")